# Downloading MODIS MCD43A2 quality and snow info

## Download data

First we have to import pymodis library and set all the needed variables

In [6]:
import os
import sys
import glob
from pathlib import Path
from IPython.core.display import Image
from pymodis import downmodis
from pymodis import parsemodis
from pymodis.convertmodis_gdal import convertModisGDAL
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tifffile as tiff

# Your Earthdata login info
Earthdata_user = ""
Earthdata_password ""

In [7]:
# destination folder
dest = "/home/cluster/eplekh/scratch/qa2/raw/"
output_dest = "/home/cluster/eplekh/scratch/qa2/mosaics/"
Path(output_dest).mkdir(parents=True, exist_ok=True)
Path(dest).mkdir(parents=True, exist_ok=True)
# tiles to download
tiles = ["h09v02"] + ["h" + str(i) + "v02" for i in range(10,27)] + \
["h" + str(i) + "v01" for i in range(12,24)] + ["h" + str(i) + "v00" for i in range(15,21)]
# number of day to download
delta = 1

### Download HDFs

In [10]:
from pymodis.convertmodis_gdal import createMosaicGDAL
# quality flags for the bands of interest from the MCD43A2 product
subset = [0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1]

In [ ]:
%%time 
for year in range(2000, 2022): 
    print(year)
    day = str(year) + "-07-15"
    #dest = "../data/modis/"+ str(year) + "/"
    dest = "/home/cluster/eplekh/scratch/qa2/raw/"+ str(year) + "/"
    Path(dest).mkdir(parents=True, exist_ok=True)
    
    output_pref = os.path.join(output_dest, str(year) + '.mosaic')
    
    modisDown = downmodis.downModis(destinationFolder=dest, tiles=tiles, today=day, delta=delta,
                               product='MCD43A2.061', path='MOTA', user = Earthdata_user, password = Earthdata_password)
    %time modisDown.connect()
    %time modisDown.downloadsAllDay()
    files = glob.glob(os.path.join(dest, 'MCD43A2*.hdf'))
    print(len(files))
    
    %time mosaic = createMosaicGDAL(files, subset, 'GTiff')
    mosaic.write_vrt(output_pref)

In [13]:
%%time 
subset = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] # downoading only snow flag
for year in range(2000, 2022): 
    print(year)
    day = str(year) + "-07-15"
    dest = "/home/cluster/eplekh/scratch/qa2/raw/"+ str(year) + "/"
   
    output_pref = os.path.join(output_dest, str(year) + '.mosaic')
 
    files = glob.glob(os.path.join(dest, 'MCD43A2*.hdf'))
    print(len(files))
    
    %time mosaic = createMosaicGDAL(files, subset, 'GTiff')
    mosaic.write_vrt(output_pref)

2000
34
CPU times: user 318 ms, sys: 2.27 s, total: 2.59 s
Wall time: 2.59 s
The VRT mosaic file /home/cluster/eplekh/scratch/qa2/mosaics/2000.mosaic has been created
2001
33
CPU times: user 300 ms, sys: 2.15 s, total: 2.45 s
Wall time: 2.46 s
The VRT mosaic file /home/cluster/eplekh/scratch/qa2/mosaics/2001.mosaic has been created
2002
8
CPU times: user 86.5 ms, sys: 580 ms, total: 666 ms
Wall time: 666 ms
The VRT mosaic file /home/cluster/eplekh/scratch/qa2/mosaics/2002.mosaic has been created
2003
6
CPU times: user 60.2 ms, sys: 544 ms, total: 604 ms
Wall time: 604 ms
The VRT mosaic file /home/cluster/eplekh/scratch/qa2/mosaics/2003.mosaic has been created
2004
5
CPU times: user 38.3 ms, sys: 296 ms, total: 334 ms
Wall time: 334 ms
The VRT mosaic file /home/cluster/eplekh/scratch/qa2/mosaics/2004.mosaic has been created
2005
3
CPU times: user 29.5 ms, sys: 196 ms, total: 226 ms
Wall time: 226 ms
The VRT mosaic file /home/cluster/eplekh/scratch/qa2/mosaics/2005.mosaic has been create

In [14]:
%%time
vrtfiles = glob.glob(os.path.join(output_dest, '*Albedo.vrt'))
from pymodis.convertmodis_gdal import convertModisGDAL
# [daily temp, quality for daily, not used, not used, nightly temp, quality for nightly]
wkt = "+proj=laea +lat_0=90 +lon_0=180 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs"

for f in vrtfiles:
    base = os.path.basename(f).replace('.vrt', '_vrt')
    output = os.path.join(output_dest, base)
    convertsingle = convertModisGDAL(hdfname=f, prefix=output, subset= subset, res=500, wkt = wkt, vrt=True)
    convertsingle.run_vrt_separated() 

Layer /home/cluster/eplekh/scratch/qa2/mosaics/2000.mosaic_Snow_BRDF_Albedo.vrt reprojected
Dataset '/home/cluster/eplekh/scratch/qa2/mosaics/2000.mosaic_Snow_BRDF_Albedo.vrt' reprojected
Layer /home/cluster/eplekh/scratch/qa2/mosaics/2010.mosaic_Snow_BRDF_Albedo.vrt reprojected
Dataset '/home/cluster/eplekh/scratch/qa2/mosaics/2010.mosaic_Snow_BRDF_Albedo.vrt' reprojected
Layer /home/cluster/eplekh/scratch/qa2/mosaics/2009.mosaic_Snow_BRDF_Albedo.vrt reprojected
Dataset '/home/cluster/eplekh/scratch/qa2/mosaics/2009.mosaic_Snow_BRDF_Albedo.vrt' reprojected
Layer /home/cluster/eplekh/scratch/qa2/mosaics/2002.mosaic_Snow_BRDF_Albedo.vrt reprojected
Dataset '/home/cluster/eplekh/scratch/qa2/mosaics/2002.mosaic_Snow_BRDF_Albedo.vrt' reprojected
Layer /home/cluster/eplekh/scratch/qa2/mosaics/2003.mosaic_Snow_BRDF_Albedo.vrt reprojected
Dataset '/home/cluster/eplekh/scratch/qa2/mosaics/2003.mosaic_Snow_BRDF_Albedo.vrt' reprojected
Layer /home/cluster/eplekh/scratch/qa2/mosaics/2021.mosaic_S

In [5]:
%%time
vrtfiles = glob.glob(os.path.join(output_dest, '*.vrt'))
from pymodis.convertmodis_gdal import convertModisGDAL
# [daily temp, quality for daily, not used, not used, nightly temp, quality for nightly]
wkt = "+proj=laea +lat_0=90 +lon_0=180 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs"

for f in vrtfiles:
    base = os.path.basename(f).replace('.vrt', '_vrt')
    output = os.path.join(output_dest, base)
    convertsingle = convertModisGDAL(hdfname=f, prefix=output, subset= subset, res=500, wkt = wkt, vrt=True)
    convertsingle.run_vrt_separated() 

Layer /home/cluster/eplekh/scratch/qa2/mosaics/2001.mosaic_BRDF_Albedo_Band_Quality_Band3.vrt reprojected
Dataset '/home/cluster/eplekh/scratch/qa2/mosaics/2001.mosaic_BRDF_Albedo_Band_Quality_Band3.vrt' reprojected
Layer /home/cluster/eplekh/scratch/qa2/mosaics/2006.mosaic_BRDF_Albedo_Band_Quality_Band6.vrt reprojected
Dataset '/home/cluster/eplekh/scratch/qa2/mosaics/2006.mosaic_BRDF_Albedo_Band_Quality_Band6.vrt' reprojected
Layer /home/cluster/eplekh/scratch/qa2/mosaics/2017.mosaic_BRDF_Albedo_Band_Quality_Band5.vrt reprojected
Dataset '/home/cluster/eplekh/scratch/qa2/mosaics/2017.mosaic_BRDF_Albedo_Band_Quality_Band5.vrt' reprojected
Layer /home/cluster/eplekh/scratch/qa2/mosaics/2019.mosaic_BRDF_Albedo_Band_Quality_Band6.vrt reprojected
Dataset '/home/cluster/eplekh/scratch/qa2/mosaics/2019.mosaic_BRDF_Albedo_Band_Quality_Band6.vrt' reprojected
Layer /home/cluster/eplekh/scratch/qa2/mosaics/2008.mosaic_BRDF_Albedo_Band_Quality_Band6.vrt reprojected
Dataset '/home/cluster/eplekh/

### Crop all files to CAVM

In [15]:
import rioxarray as rxr
import xarray as xr
import geopandas as gpd
from shapely.geometry import mapping

In [16]:
cavm_path = "/home/cluster/eplekh/data/cavm/shape/cp_veg_la.shp"
crop_extent = gpd.read_file(cavm_path)

In [17]:
clip_path = "/home/cluster/eplekh/scratch/qa2/cropped_mosaics_sw/"
os.mkdir(clip_path)
tiffiles = glob.glob(os.path.join(output_dest, '*.tif'))
len(tiffiles)

176

In [18]:
for i in range(1,8):
    clip_path = "/home/cluster/eplekh/scratch/qa2/cropped_mosaics_band" + str(i) + "/"
    os.mkdir(clip_path)
    for year in range(2000, 2022):
        print(year)
        tif_path = output_dest+str(year)+".mosaic_BRDF_Albedo_Band_Quality_Band" + str(i) + "_vrt.tif"
        modis_year = rxr.open_rasterio(tif_path, masked=True).squeeze()
        %time modis_year_clipped = modis_year.rio.clip(crop_extent.geometry.apply(mapping), crop_extent.crs)
        # Write the data to a new geotiff file
        modis_year_clipped.rio.to_raster(clip_path + str(year) + "_band" + str(i) + ".tif")

2000
CPU times: user 2.64 s, sys: 2.45 s, total: 5.09 s
Wall time: 5.89 s
2001
CPU times: user 2.54 s, sys: 628 ms, total: 3.17 s
Wall time: 3.98 s
2002
CPU times: user 2.59 s, sys: 617 ms, total: 3.21 s
Wall time: 3.93 s
2003
CPU times: user 2.24 s, sys: 461 ms, total: 2.7 s
Wall time: 3.44 s
2004
CPU times: user 2.35 s, sys: 573 ms, total: 2.92 s
Wall time: 3.52 s
2005
CPU times: user 1.93 s, sys: 316 ms, total: 2.25 s
Wall time: 2.73 s
2006
CPU times: user 2.85 s, sys: 925 ms, total: 3.78 s
Wall time: 4.5 s
2007
CPU times: user 2.33 s, sys: 591 ms, total: 2.92 s
Wall time: 10.5 s
2008
CPU times: user 2.58 s, sys: 650 ms, total: 3.23 s
Wall time: 3.99 s
2009
CPU times: user 2.58 s, sys: 646 ms, total: 3.23 s
Wall time: 4.05 s
2010
CPU times: user 2.47 s, sys: 653 ms, total: 3.12 s
Wall time: 5.65 s
2011
CPU times: user 2.66 s, sys: 626 ms, total: 3.29 s
Wall time: 10.6 s
2012
CPU times: user 2.5 s, sys: 668 ms, total: 3.17 s
Wall time: 8.22 s
2013
CPU times: user 2.65 s, sys: 632 ms,

In [19]:
clip_path = "/home/cluster/eplekh/scratch/qa2/cropped_mosaics_snowflag/"
os.mkdir(clip_path)
for year in range(2000, 2022):
    print(year)
    tif_path = output_dest+str(year)+".mosaic_Snow_BRDF_Albedo_vrt.tif"
    modis_year = rxr.open_rasterio(tif_path, masked=True).squeeze()
    %time modis_year_clipped = modis_year.rio.clip(crop_extent.geometry.apply(mapping), crop_extent.crs)
    # Write the data to a new geotiff file
    modis_year_clipped.rio.to_raster(clip_path + str(year) + "_snowflag.tif")

2000
CPU times: user 2.54 s, sys: 601 ms, total: 3.14 s
Wall time: 4.06 s
2001
CPU times: user 2.67 s, sys: 610 ms, total: 3.28 s
Wall time: 4.13 s
2002
CPU times: user 2.51 s, sys: 554 ms, total: 3.07 s
Wall time: 3.89 s
2003
CPU times: user 2.29 s, sys: 531 ms, total: 2.82 s
Wall time: 3.5 s
2004
CPU times: user 2.33 s, sys: 484 ms, total: 2.81 s
Wall time: 3.5 s
2005
CPU times: user 1.92 s, sys: 353 ms, total: 2.27 s
Wall time: 2.79 s
2006
CPU times: user 2.42 s, sys: 510 ms, total: 2.93 s
Wall time: 3.77 s
2007
CPU times: user 2.44 s, sys: 530 ms, total: 2.97 s
Wall time: 3.89 s
2008
CPU times: user 2.52 s, sys: 642 ms, total: 3.16 s
Wall time: 3.98 s
2009
CPU times: user 2.62 s, sys: 645 ms, total: 3.27 s
Wall time: 4.25 s
2010
CPU times: user 2.53 s, sys: 572 ms, total: 3.1 s
Wall time: 3.88 s
2011
CPU times: user 2.66 s, sys: 630 ms, total: 3.29 s
Wall time: 4.15 s
2012
CPU times: user 2.45 s, sys: 668 ms, total: 3.12 s
Wall time: 3.94 s
2013
CPU times: user 2.63 s, sys: 631 ms,